# Multi Fabric - Commissioning and Interactions

<a href="http://35.236.121.59/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fproject-chip%2Fconnectedhomeip&urlpath=lab%2Ftree%2Fconnectedhomeip%2Fdocs%2Fguides%2Frepl%2FMatter%2520-%2520Multi%2520Fabric%2520Commissioning.ipynb&branch=master">
<img src="https://i.ibb.co/hR3yWsC/launch-playground.png" alt="drawing" width="130"/>
</a>
<br></br>

This walks through creating multiple controllers on multiple fabrics, using those controllers to commission a target onto those fabrics and finally, interacting with them using the interaction model.

## FabricAdmins and Controllers

The `FabricAdmin` class (present in the `chip.FabricAdmin` package) is responsible for administering a fabric. It houses the Fabric ID and Index, as well as an RCAC and ICAC that provides the certificate material grounding that fabric.

The `FabricAdmin` can be used to vend `ChipDeviceController` objects that represent a controller instance with a specific identity grounded in the admin's fabric. This controller can then be used to commission and interact with devices.

## Clear Persisted Storage

Let's clear out our persisted storage (if one exists) to start from a clean slate.

In [1]:
import os, subprocess

if os.path.isfile('/tmp/repl-storage.json'):
    os.remove('/tmp/repl-storage.json')

# So that the all-clusters-app won't boot with stale prior state.    
os.system('rm -rf /tmp/chip_*')

## Initialization

Let's first begin by setting up by importing some key modules that are needed to make it easier for us to interact with the Matter stack.

`ChipReplStartup.py` is run within the global namespace. This results in all of its imports being made available here.

> **NOTE**: _This is not needed if you launch the REPL from the command-line._

In [2]:
import chip.native
import pkgutil
module = pkgutil.get_loader('chip.ChipReplStartup')
%run {module.path}

──────────────────────────────────────── Matter REPL ────────────────────────────────────────

    
            Welcome to the Matter Python REPL!
    
            For help, please type matterhelp()
    
            To get more information on a particular object/class, you can pass
            that into matterhelp() as well.
    
            

─────────────────────────────────────────────────────────────────────────────────────────────

2022-01-25 16:58:57 johnsj-macbookpro1.roam.corp.google.com root[27921] ERROR [Errno 2] No such file or directory: '/tmp/repl-storage.json'
2022-01-25 16:58:57 johnsj-macbookpro1.roam.corp.google.com root[27921] WARNING Could not load configuration from /tmp/repl-storage.json - resetting configuration...


No previous fabric admins discovered in persistent storage - creating a new one...

New FabricAdmin: FabricId: 1(1)


Creating default device controller on fabric 1...

Allocating new controller with FabricId: 1(1), NodeId: 1


Default CHIP Device Controller has been initialized to manage fabricAdmins[0], and is 
available as devCtrl

At startup, the REPL will attempt to find any previously configured fabrics stored in persisted storage. If it can't find any (as is the case here), it will construct a default `FabricAdmin` object on Fabric 1 (Index 1) as well as construct a device controller (`devCtrl`) on that fabric.

In [3]:
fabricAdmins

[
│   <chip.FabricAdmin.FabricAdmin object at 0x10a68dee0>
]

In [4]:
devCtrl

<chip.ChipDeviceCtrl.ChipDeviceController object at 0x10a6c6040>

### Commission onto Fabric 1

#### Launch Server

Let's launch an instance of the `chip-all-clusters-app`.

In [5]:
import time, os
import subprocess
os.system('pkill -f chip-all-clusters-app')
time.sleep(1)

# The location of the all-clusters-app in the cloud playground is one level higher - adjust for this by testing for file presence.
if (os.path.isfile('../../../out/debug/chip-all-clusters-app')):
    appPath = '../../../out/debug/chip-all-clusters-app'
else:
    appPath = '../../../../out/debug/chip-all-clusters-app'

process = subprocess.Popen(appPath, stdout=subprocess.DEVNULL)
time.sleep(1)

#### Commission Target

Commission the target onto Fabric 1 using the default device controller instance with a NodeId of 1.

In [6]:
devCtrl.CommissionIP(b'127.0.0.1', 20202021, 2)

2022-01-25 16:59:00 johnsj-macbookpro1.roam.corp.google.com chip.CTL[27921] ERROR Unable to find country code, defaulting to WW
2022-01-25 16:59:00 johnsj-macbookpro1.roam.corp.google.com chip.SC[27921] ERROR The device does not support GetClock_RealTimeMS() API. This will eventually result in CASE session setup failures.


Node address has been updated
Commissioning complete


True

### Read OpCreds Cluster

Read out the OpCreds cluster to confirm membership into Fabric 1.

In [7]:
await devCtrl.ReadAttribute(2, [(Clusters.OperationalCredentials.Attributes.FabricsList)], fabricFiltered=False)

{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: {
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.FabricsList'>: [
│   │   │   │   FabricDescriptor(
│   │   │   │   │   fabricIndex=1,
│   │   │   │   │   rootPublicKey=b'\x04\xa2\x88}\xbd\xdb\xd4\xbc^U\xa3\xcef\xf7\x92\xbe\xa7X;E\xb8-Y\x12^$\xe0\x1bS\xf9\x9a\x8f\xacvj\xfa!&m6\x00\xe3\xd1\x85o\x1b\x9eH\xabC?\xd8\xec\x08lH\xb5X\x1f:u\xb0\xfbj\x98',
│   │   │   │   │   vendorId=27168,
│   │   │   │   │   fabricId=1,
│   │   │   │   │   nodeId=2,
│   │   │   │   │   label=''
│   │   │   │   )
│   │   │   ]
│   │   }
│   }
}

### Commission onto Fabric 2

#### Create new FabricAdmin

In [8]:
import chip.FabricAdmin as FabricAdmin
fabric2 = FabricAdmin.FabricAdmin(fabricId = 2, fabricIndex = 2)

New FabricAdmin: FabricId: 2(2)


Here's a brief peek at the JSON data that is in the persisted storage file.

In [9]:
builtins.chipStack.GetStorageManager().jsonData

{
│   'repl-config': {
│   │   'fabricAdmins': {
│   │   │   '1': {
│   │   │   │   'fabricId': 1
│   │   │   },
│   │   │   '2': {
│   │   │   │   'fabricId': 2
│   │   │   }
│   │   }
│   },
│   'sdk-config': {
│   │   'ExampleOpCredsCAKey1': 'BKKIfb3b1LxeVaPOZveSvqdYO0W4LVkSXiTgG1P5mo+sdmr6ISZtNgDj0YVvG55Iq0M/2OwIbEi1WB86dbD7apgpj13h8fueMKMfErM65jMAsAPBcPEhnVc3JJ6m7K4Qq0ORAwBgAABhAAAAAAAAAA==',
│   │   'ExampleOpCredsICAKey1': 'BCwMgmSo2C1vhjBR642H5nieC/jJManFAW54QhdovJhdySYZn96XJoBNyouhoGf9rn0SipgIhmvHPcPqDrDwcAXiQyB87caIuz5DJ3v7KHHLquGmdv5pNpZ4IhqDomt/2kORAwBgAABhAAAAAAAAAA==',
│   │   'ExampleCAIntermediateCert1': 'MIIB2TCCAYCgAwIBAgIBADAKBggqhkjOPQQDAjBEMSAwHgYKKwYBBAGConwBBAwQMDAwMDAwMDAwMDAwMDAwMDEgMB4GCisGAQQBgqJ8AQUMEDAwMDAwMDAwMDAwMDAwMDEwHhcNMjEwMTAxMDAwMDAwWhcNMzAxMjMwMDAwMDAwWjBEMSAwHgYKKwYBBAGConwBAwwQMDAwMDAwMDAwMDAwMDAwMTEgMB4GCisGAQQBgqJ8AQUMEDAwMDAwMDAwMDAwMDAwMDEwWTATBgcqhkjOPQIBBggqhkjOPQMBBwNCAAQsDIJkqNgtb4YwUeuNh+Z4ngv4yTGpxQFueEIXaLyYXckmGZ/elyaATcqLoaBn/a59EoqYCIZrxz3D6g6w8HAFo2MwYTAPBgNVHRMBAf8EBTADAQH/MA4GA1UdDwEB/wQEAwIBBjAdBgNVHQ4EFgQUcPz5CkZJujm2U0jL0G1lZwoJXD0wHwYDVR0jBBgwFoAUfvAKJyeJwjao2nq+Q3ayf27KXrUwCgYIKoZIzj0EAwIDRwAwRAIgEaX5gBqc5b8eswqDDlc2IQ+wLKfbsONC3FzsuwLCBNkCIAyn32oaFV0YCN9GeiUpYvqbypnMWQGnffF85v7wKhBX',
│   │   'ExampleCARootCert1': 'MIIB2jCCAYCgAwIBAgIBADAKBggqhkjOPQQDAjBEMSAwHgYKKwYBBAGConwBBAwQMDAwMDAwMDAwMDAwMDAwMDEgMB4GCisGAQQBgqJ8AQUMEDAwMDAwMDAwMDAwMDAwMDEwHhcNMjEwMTAxMDAwMDAwWhcNMzAxMjMwMDAwMDAwWjBEMSAwHgYKKwYBBAGConwBBAwQMDAwMDAwMDAwMDAwMDAwMDEgMB4GCisGAQQBgqJ8AQUMEDAwMDAwMDAwMDAwMDAwMDEwWTATBgcqhkjOPQIBBggqhkjOPQMBBwNCAASiiH2929S8XlWjzmb3kr6nWDtFuC1ZEl4k4BtT+ZqPrHZq+iEmbTYA49GFbxueSKtDP9jsCGxItVgfOnWw+2qYo2MwYTAPBgNVHRMBAf8EBTADAQH/MA4GA1UdDwEB/wQEAwIBBjAdBgNVHQ4EFgQUfvAKJyeJwjao2nq+Q3ayf27KXrUwHwYDVR0jBBgwFoAUfvAKJyeJwjao2nq+Q3ayf27KXrUwCgYIKoZIzj0EAwIDSAAwRQIgMzvVbqCxT+xJOpxN9HUSYZYdExmkRLN2Jl9vLijLc+sCIQDAxLgwwuxGNPtWJ/dY4kGqpWwNNbq9pEkmQ//t2KhfBg==',
│   │   'ExampleOpCredsCAKey2': 'BJ1gkAlp0ZrmBkQiko2ces/p86N1qCB30y+tdT5HrWEBqgPh6SVNnZfe5ZMjzJZFp/J1K+7MSDF7QeAgKrQYwcAcV1G9fnqw9anHoj+EW2k9xqw0b4bWXE9f2xKfWWb/ykCSAwBgAABhAAAAAAAAAA==',
│   │   'ExampleOpCredsICAKey2': 'BMktytxSOcWsbMcZ6WrjsODvTRKcLWZjo8bt8CDUqfhoHLFqrmahTQnH4rY8gjRpKRBw/Yw2YpEodcaLyC1V+cC2VvZrLqLVW8uRNPkVxcojVHAXYAGCRp6CW4kQ+zBtNUCSAwBgAABhAAAAAAAAAA=='
│   }
}

In [10]:
devCtrl2 = fabric2.NewController()

Allocating new controller with FabricId: 2(2), NodeId: 1


#### Open Commissioning Window

In [11]:
await devCtrl.SendCommand(2, 0, Clusters.AdministratorCommissioning.Commands.OpenBasicCommissioningWindow(180))

In [12]:
devCtrl2.CommissionIP(b'127.0.0.1', 20202021, 2)

2022-01-25 16:59:00 johnsj-macbookpro1.roam.corp.google.com chip.CTL[27921] ERROR Unable to find country code, defaulting to WW
2022-01-25 16:59:00 johnsj-macbookpro1.roam.corp.google.com chip.SC[27921] ERROR The device does not support GetClock_RealTimeMS() API. This will eventually result in CASE session setup failures.


Node address has been updated
Commissioning complete


True

### Read OpCreds Cluster

Read out the OpCreds cluster to confirm membership into Fabric 2.

In [13]:
await devCtrl2.ReadAttribute(2, [(Clusters.OperationalCredentials.Attributes.FabricsList)], fabricFiltered=False)

{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: {
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.FabricsList'>: [
│   │   │   │   FabricDescriptor(
│   │   │   │   │   fabricIndex=1,
│   │   │   │   │   rootPublicKey=b'\x04\xa2\x88}\xbd\xdb\xd4\xbc^U\xa3\xcef\xf7\x92\xbe\xa7X;E\xb8-Y\x12^$\xe0\x1bS\xf9\x9a\x8f\xacvj\xfa!&m6\x00\xe3\xd1\x85o\x1b\x9eH\xabC?\xd8\xec\x08lH\xb5X\x1f:u\xb0\xfbj\x98',
│   │   │   │   │   vendorId=27168,
│   │   │   │   │   fabricId=1,
│   │   │   │   │   nodeId=2,
│   │   │   │   │   label=''
│   │   │   │   ),
│   │   │   │   FabricDescriptor(
│   │   │   │   │   fabricIndex=2,
│   │   │   │   │   rootPublicKey=b'\x04\x9d`\x90\ti\xd1\x9a\xe6\x06D"\x92\x8d\x9cz\xcf\xe9\xf3\xa3u\xa8 w\xd3/\xadu>G\xada\x01\xaa\x03\xe1\xe9%M\x9d\x97\xde\xe5\x93#\xcc\x96E\xa7\xf2u+\xee\xccH1{A\xe0 *\xb4\x18\xc1\xc0',
│   │   │   │   │   vendorId=28544,
│   │   │   │   │   fabricId=2,
│   │   │   │   │   nodeId=2,
│   │   │   │   │   label=''
│   │   │   │   )
│   │   │   ]
│   │   }
│   }
}

## Relaunch REPL

Let's simulate re-launching the REPL to show-case the capabilities of the persistence storage and its mechanics.

In [14]:
import chip.native
import pkgutil
module = pkgutil.get_loader('chip.ChipReplStartup')
%run {module.path}

──────────────────────────────────────── Matter REPL ────────────────────────────────────────

    
            Welcome to the Matter Python REPL!
    
            For help, please type matterhelp()
    
            To get more information on a particular object/class, you can pass
            that into matterhelp() as well.
    
            

─────────────────────────────────────────────────────────────────────────────────────────────

Restoring FabricAdmin from storage to manage FabricId 1, FabricIndex 1...

New FabricAdmin: FabricId: 1(1)


...

New FabricAdmin: FabricId: 2(2)


Fabric Admins have been loaded and are available at fabricAdmins

Creating default device controller on fabric 1...

Allocating new controller with FabricId: 1(1), NodeId: 1


Default CHIP Device Controller has been initialized to manage fabricAdmins[0], and is 
available as devCtrl

The REPL has now loaded the two fabrics that were created in the previous session into the `fabricAdmins` variable. It has also created a default controller on the first fabric in that list (Fabric 1) as `devCtrl`.

### Establish CASE and Read OpCreds

To prove that we do indeed have two distinct fabrics and controllers on each fabric, let's go ahead and update the label of each fabric. To do so, you'd need to successfully establish a CASE session through a controller on the respective fabric, and call the 'UpdateLabel' command.

Underneath the covers, each device controller will do operational discovery of the NodeId being read and establish a CASE session before issuing the IM interaction.

In [15]:
await devCtrl.SendCommand(2, 0, Clusters.OperationalCredentials.Commands.UpdateFabricLabel("Fabric1Label"))
await devCtrl.ReadAttribute(2, [(Clusters.OperationalCredentials.Attributes.FabricsList)], fabricFiltered=False)

2022-01-25 16:59:00 johnsj-macbookpro1.roam.corp.google.com chip.SC[27921] ERROR The device does not support GetClock_RealTimeMS() API. This will eventually result in CASE session setup failures.


Node address has been updated
Established CASE with Device


{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: {
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.FabricsList'>: [
│   │   │   │   FabricDescriptor(
│   │   │   │   │   fabricIndex=1,
│   │   │   │   │   rootPublicKey=b'\x04\xa2\x88}\xbd\xdb\xd4\xbc^U\xa3\xcef\xf7\x92\xbe\xa7X;E\xb8-Y\x12^$\xe0\x1bS\xf9\x9a\x8f\xacvj\xfa!&m6\x00\xe3\xd1\x85o\x1b\x9eH\xabC?\xd8\xec\x08lH\xb5X\x1f:u\xb0\xfbj\x98',
│   │   │   │   │   vendorId=27168,
│   │   │   │   │   fabricId=1,
│   │   │   │   │   nodeId=2,
│   │   │   │   │   label='Fabric1Label'
│   │   │   │   ),
│   │   │   │   FabricDescriptor(
│   │   │   │   │   fabricIndex=2,
│   │   │   │   │   rootPublicKey=b'\x04\x9d`\x90\ti\xd1\x9a\xe6\x06D"\x92\x8d\x9cz\xcf\xe9\xf3\xa3u\xa8 w\xd3/\xadu>G\xada\x01\xaa\x03\xe1\xe9%M\x9d\x97\xde\xe5\x93#\xcc\x96E\xa7\xf2u+\xee\xccH1{A\xe0 *\xb4\x18\xc1\xc0',
│   │   │   │   │   vendorId=28544,
│   │   │   │   │   fabricId=2,
│   │   │   │   │   nodeId=2,
│   │   │   │   │   label=''
│   │   │   │   )
│   │   │   ]
│   │   }
│   }
}

Instantiate a controller on fabric 2 and use it to read out the op creds from that fabric.

In [16]:
devCtrl2 = fabricAdmins[1].NewController()
await devCtrl2.SendCommand(2, 0, Clusters.OperationalCredentials.Commands.UpdateFabricLabel("Fabric2Label"))
await devCtrl2.ReadAttribute(2, [(Clusters.OperationalCredentials.Attributes.FabricsList)], fabricFiltered=False)

2022-01-25 16:59:00 johnsj-macbookpro1.roam.corp.google.com chip.SC[27921] ERROR The device does not support GetClock_RealTimeMS() API. This will eventually result in CASE session setup failures.


Allocating new controller with FabricId: 2(2), NodeId: 1
Node address has been updated
Established CASE with Device


{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: {
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.FabricsList'>: [
│   │   │   │   FabricDescriptor(
│   │   │   │   │   fabricIndex=1,
│   │   │   │   │   rootPublicKey=b'\x04\xa2\x88}\xbd\xdb\xd4\xbc^U\xa3\xcef\xf7\x92\xbe\xa7X;E\xb8-Y\x12^$\xe0\x1bS\xf9\x9a\x8f\xacvj\xfa!&m6\x00\xe3\xd1\x85o\x1b\x9eH\xabC?\xd8\xec\x08lH\xb5X\x1f:u\xb0\xfbj\x98',
│   │   │   │   │   vendorId=27168,
│   │   │   │   │   fabricId=1,
│   │   │   │   │   nodeId=2,
│   │   │   │   │   label='Fabric1Label'
│   │   │   │   ),
│   │   │   │   FabricDescriptor(
│   │   │   │   │   fabricIndex=2,
│   │   │   │   │   rootPublicKey=b'\x04\x9d`\x90\ti\xd1\x9a\xe6\x06D"\x92\x8d\x9cz\xcf\xe9\xf3\xa3u\xa8 w\xd3/\xadu>G\xada\x01\xaa\x03\xe1\xe9%M\x9d\x97\xde\xe5\x93#\xcc\x96E\xa7\xf2u+\xee\xccH1{A\xe0 *\xb4\x18\xc1\xc0',
│   │   │   │   │   vendorId=28544,
│   │   │   │   │   fabricId=2,
│   │   │   │   │   nodeId=2,
│   │   │   │   │   label='Fabric2Label'
│   │   │   │   )
│   │   │   ]
│   │   }
│   }
}

In [17]:
devCtrl2.Shutdown()